# Проект по SQL



Цель исследования: проанализировать базу данных сервиса для чтения книг по подписке. Исследуем данные и ответим на ключевые вопросы:
- Сколько книг вышло после 1 января 2000 года;
- Посчитаем количество обзоров и среднюю оценку для каждой книги;
- Определим издательство, которое выпустило наибольшее число книг толще
50 страниц — так вы исключите из анализа брошюры;
- Определим автора с самой высокой средней оценкой книг (учитывать
только книги с 50 и более оценками);
- Посчитаем среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.

## Знакомство с данными

Импортирем библиотеки и получим доступ к базе данных.

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [2]:
# устанавливаем параметры
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
  '''Открываем соединение, получаем данные из sql, закрываем соединение'''
  with engine.connect() as con:
    return pd.read_sql(sql=sa.text(query), con = con)

Выведем первые строки каждой таблицы и посчитаем строки в каждой из них

### books

Таблица содержит данные о книгах

In [4]:
# формируем запрос и выводим данные
query = '''

SELECT *
FROM books
LIMIT 5

'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

In [5]:
query = '''

SELECT COUNT(*)
FROM books

'''
get_sql_data(query)

,count
0,1000


В таблице 1000 строк.

### authors

Таблица содержит данные об авторах

In [6]:
query = '''

SELECT *
FROM authors
LIMIT 5

'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


- author_id — идентификатор автора;
- author — имя автора.

In [7]:
query = '''

SELECT COUNT(*)
FROM authors

'''
get_sql_data(query)

,count
0,636


В таблице данные о 636 авторах

### publishers

Таблица содержит данные об издательствах

In [8]:
query = '''

SELECT *
FROM publishers
LIMIT 5

'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


- publisher_id — идентификатор издательства;
- publisher — название издательства;

In [9]:
query = '''

SELECT COUNT(*)
FROM publishers

'''
get_sql_data(query)

,count
0,340


В таблице записаны данные о 340 издательствах

### ratings

Таблица содержит данные о пользовательских оценках книг

In [10]:
query = '''

SELECT *
FROM ratings
LIMIT 5

'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

In [11]:
query = '''

SELECT
COUNT(*)
FROM ratings

'''
get_sql_data(query)

,count
0,6456


В таблице записано 6456 строк

### reviews

Таблица содержит данные о пользовательских обзорах на книги

In [12]:
query = '''

SELECT *
FROM reviews
LIMIT 5

'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

In [13]:
query = '''

SELECT COUNT(*)
FROM reviews

'''
get_sql_data(query)

,count
0,2793


В таблице сохранены данные о 2793 обзорах

## Количество книг вышедших после 1 января 2000 года

Посчитаем, сколько книг вышло после 1 января 2000 года

In [14]:
query = '''

SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01';

'''
get_sql_data(query)

,count
0,819


После 1 января 2000 года вышло 819 книг.

## Количество обзоров и средняя оценка книг

Для каждой книги посчитаем количество обзоров и среднюю оценку

In [15]:
query = '''

SELECT books.book_id AS book_id,
       books.title AS book_title,
       COUNT(DISTINCT reviews.review_id) AS count_reviews,
       AVG(ratings.rating) AS avg_rating
FROM books
INNER JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY books.title,
         books.book_id
ORDER BY avg_rating DESC;

'''
get_sql_data(query)

,book_id,book_title,count_reviews,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


In [16]:
query = '''

SELECT books.book_id AS book_id,
       books.title AS book_title,
       COUNT(DISTINCT reviews.review_id) AS count_reviews,
       AVG(ratings.rating) AS avg_rating
FROM books
INNER JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY books.title,
         books.book_id
ORDER BY count_reviews DESC;

'''
get_sql_data(query)

,book_id,book_title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Для каждой книги мы посчитали количество рецензий и среднюю пользовательскую оценку, большое количсевто книг с высоким рейтингом, самый низкий рейтинг у книги Джоди Пиколт: Забрать любовь.

Самое большое количество обзоров на книгу Стефани Майер: Сумерки.

## Издательство, которое выпустило наибольшее число книг

Определим издательство, которое выпустило наибольшее число книг толще
50 страниц (исключим из анализа брошюры)

In [17]:
query = '''

SELECT publishers.publisher,
       COUNT(books.book_id) as count_books
FROM publishers
JOIN books ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher_id
ORDER BY count_books DESC
LIMIT 1;

'''
get_sql_data(query)

,publisher,count_books
0,Penguin Books,42


Издательство, которое выпустило самое большое количество книг: Penguin Books	(42 книги более 50 страниц). Британское издательство, основанное в 1935 году в Лондоне.

## Автор с самой высокой средней оценкой книг

Определим автора с самой высокой средней оценкой книг — учитываем
только книги с 50 и более оценками.

In [18]:
query = '''

SELECT a.author AS author,
       AVG(a.avg_rate) AS avg_rate
FROM (SELECT authors.author AS author,
             b.book_id AS book_id,
             AVG(r.rating) AS avg_rate,
             COUNT(r.rating) AS count_rating
      FROM authors
      INNER JOIN books AS b ON b.author_id = authors.author_id
      INNER JOIN ratings AS r ON r.book_id = b.book_id
      GROUP BY authors.author,
               b.book_id
      HAVING COUNT(r.rating) >= 50) AS a
GROUP BY author
ORDER BY avg_rate DESC
LIMIT 1;

'''
get_sql_data(query)

,author,avg_rate
0,J.K. Rowling/Mary GrandPré,4.283844


Самый высокий рейтинг среди авторов с более чем 50 оценок - у британской писательницы Джоан Роулинг.

## Среднее количество обзоров от пользователей

Посчитаем среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.


In [19]:
query = '''

SELECT AVG(review.review_cnt) AS avg_review
FROM (SELECT COUNT(review_id) AS review_cnt
      FROM reviews
      WHERE username
      IN (SELECT username
          FROM ratings
          GROUP BY username
          HAVING
          COUNT(ratings) > 48)
          GROUP BY username) AS review;

'''
get_sql_data(query)

,avg_review
0,24.0


В среднем пользователи, которые поставили более 48 оценок оставляют 24 обзора.

 ## Вывод

Мы проанализировали базу данных сервиса для чтения книг по подписке.

- После 1 января 2000 года вышло 819 книг;
- Посчитали количество обзоров и среднюю оценку для каждой книги и определили, что самый низкий рейтинг у книги Джоди Пиколт: Забрать любовь,
а самое большое количество обзоров на книгу Стефани Майер: Сумерки;
- Издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books;
- Автор с самой высокой средней оценкой книг, поличивший более 50 оценок - Джоан Роулинг;
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.